In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

imputedDataset = pd.read_stata("/Users/burke/Documents/research/bpCog/mcm/mcm/data/fullyImputedDataset.dta")
imputedDataset.head()
noRecentUSInmigration = imputedDataset.loc[~imputedDataset.timeInUS.isin([1,2,3])]
earlyCrossSectionOfNonImmigrants = noRecentUSInmigration.loc[noRecentUSInmigration.year==2001]
# here we just need to throw out the very young people for comparisons
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.loc[earlyCrossSectionOfNonImmigrants.age > 18 + 2]

earlyCrossSectionOfNonImmigrants['strokeDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2011.values == 1) & (earlyCrossSectionOfNonImmigrants.ucod.values==2)), 1, 0)
earlyCrossSectionOfNonImmigrants['miDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2011.values == 1) & (earlyCrossSectionOfNonImmigrants.ucod.values==1)), 1, 0)
earlyCrossSectionOfNonImmigrants['cvDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.strokeDeath.values == 1) | (earlyCrossSectionOfNonImmigrants.miDeath.values==1)), 1, 0)
earlyCrossSectionOfNonImmigrants['nonCVDeath'] = np.where(((earlyCrossSectionOfNonImmigrants.diedBy2011 ==1) & (earlyCrossSectionOfNonImmigrants.strokeDeath.values == 0) & (earlyCrossSectionOfNonImmigrants.miDeath.values==0)), 1, 0)


earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.sample(n=100000, weights=earlyCrossSectionOfNonImmigrants.WTINT2YR, replace=True)

In [2]:
import os
os.chdir("/Users/burke/Documents/research/bpCog/mcm")
from mcm.population import NHANESDirectSamplePopulation

pop = NHANESDirectSamplePopulation(n=10000, year=1999)

In [11]:
#pop.advance(1)
from mcm.outcome import OutcomeType
print(pop.calculate_age_sex_standardized_incidence(OutcomeType.MI, 2016))

                                    population  outcomeCount
lowerAgeBound upperAgeBound female                          
0             0             0          2021764             0
                            1          1933613             0
1             4             0          8163342             0
                            1          7800858             0
5             9             0         10431307             0


KeyError: 'lowerAgeBound'

In [56]:
from mcm.outcome import OutcomeType

ageStandard = pd.read_csv('/Users/burke/Documents/research/bpCog/mcm/mcm/data/us.1969_2017.19ages.adjusted.txt', header=0, names=['raw'])

In [57]:
# https://seer.cancer.gov/popdata/popdic.html
ageStandard['year'] = ageStandard['raw'].str[0:4]
ageStandard['year'] = ageStandard.year.astype(int)
# format changes in 1990...so, we'll go forward from there...
ageStandard = ageStandard.loc[ageStandard.year >= 1990]
ageStandard['state'] = ageStandard['raw'].str[4:6]
ageStandard['state'] = ageStandard['raw'].str[4:6]
ageStandard['race'] = ageStandard['raw'].str[13:14] # 1 = white, 2 = black, 3 = american indian/alaskan, 4 = asian/pacific islander
ageStandard['hispanic'] = ageStandard['raw'].str[14:15] 
ageStandard['female'] = ageStandard['raw'].str[15:16] 
ageStandard['female'] = ageStandard['female'].astype(int)
ageStandard['female'] = ageStandard['female'].replace({1 : 0, 2 : 1})
ageStandard['ageGroup'] = ageStandard['raw'].str[16:18]
ageStandard['ageGroup'] = ageStandard['ageGroup'].astype(int)
ageStandard['population'] = ageStandard['raw'].str[18:26]
ageStandard['population'] = ageStandard['population'].astype(int)
ageStandard['lowerAgeBound'] = (ageStandard.ageGroup -1)*5
ageStandard['upperAgeBound'] = (ageStandard.ageGroup * 5) - 1
ageStandard['lowerAgeBound'] = ageStandard['lowerAgeBound'].replace({-5 : 0, 0 : 1})
ageStandard['upperAgeBound'] = ageStandard['upperAgeBound'].replace({-1 : 0, 89 : 150})
ageStandardYear = ageStandard.loc[ageStandard.year==2016]
ageStandardGroupby = ageStandardYear[['female', 'population', 'lowerAgeBound', 'upperAgeBound','ageGroup']].groupby(
                                    ['ageGroup','female'])
ageStandardHeaders = ageStandardGroupby.first()[['lowerAgeBound', 'upperAgeBound']]
ageStandardPopulation = ageStandardYear[['female', 'population', 'ageGroup']].groupby(['ageGroup','female']).sum()
ageStandardCombined=ageStandardHeaders.join(ageStandardPopulation, how='inner')

In [58]:
ageStandardCombined['outcomeCount'] =  0
for person in pop._people:
    if len(person._outcomes[OutcomeType.MI]) > 0:
        ageStandardCombined.loc[((ageStandardCombined['lowerAgeBound'] <= person._age[-1]) & (ageStandardCombined['upperAgeBound'] >= person._age[-1])), 'outcomeCount'] += 1
ageStandardCombined['percentInGroup'] = ageStandardCombined['population'] / (ageStandardCombined['population'].sum())
ageStandardCombined['ageSpecificRate'] = ageStandardCombined['outcomeCount'] / ageStandardCombined['population'] * 100000
ageStandardCombined['ageSpecificContribution'] = ageStandardCombined['ageSpecificRate'] * ageStandardCombined['percentInGroup']
print(ageStandardCombined['ageSpecificContribution'].sum())

0.5114315542786807


In [52]:
ageStandardYear = ageStandard.loc[ageStandard.year==2016]
ageStandardYear = ageStandardYear[['female', 'population', 'lowerAgeBound', 'upperAgeBound','ageGroup']].groupby(
                                    ['ageGroup','female'])

In [53]:
ageStandardYear = ageStandard.loc[ageStandard.year==2016]
ageStandardGroupby = ageStandardYear[['female', 'population', 'lowerAgeBound', 'upperAgeBound','ageGroup']].groupby(
                                    ['ageGroup','female'])
ageStandardHeaders = ageStandardGroupby.first()[['lowerAgeBound', 'upperAgeBound']]
ageStandardPopulation = ageStandardYear[['female', 'population', 'ageGroup']].groupby(['ageGroup','female']).sum()
ageStandardCombined=ageStandardHeaders.join(ageStandardPopulation, how='inner')



In [54]:
ageStandardCombined.head()

lowerAgeBound  upperAgeBound  population
ageGroup female                                          
0        0                   0              0     2021764
         1                   0              0     1933613
1        0                   1              4     8163342
         1                   1              4     7800858
2        0                   5              9    10431307

In [ ]:
popSurvival = pd.DataFrame({"index" : pop._people.index, "baseAge" : [person._age[0] for person in pop._people]} )
popSurvival['ageDecile'] = pd.qcut(popSurvival.baseAge, 10)
for year in range(1,11):
    pop.advance(1)
    popSurvival['dead' + str(year)] = [person.is_dead() for person in pop._people]
    popSurvival['cvDeath' + str(year)] = [person.has_fatal_stroke() or person.has_fatal_mi() for person in pop._people]
    popSurvival['strokeDeath' + str(year)] = [person.has_fatal_stroke() for person in pop._people]
    popSurvival['miDeath' + str(year)] = [person.has_fatal_mi() for person in pop._people]
    popSurvival['nonCVDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==False) and (person.has_fatal_mi() == False) for person in pop._people]
    popSurvival['cvDeath' + str(year)] = [person.is_dead() and (person.has_fatal_stroke() ==True) or (person.has_fatal_mi() == True) for person in pop._people]
    

In [ ]:
earlyCrossSectionOfNonImmigrants.diedBy2011.value_counts(normalize=True)

In [ ]:
# on UCOD 1 = MI, 2=stroke, 3 = other cause of death
earlyCrossSectionOfNonImmigrants.ucod.value_counts(normalize=True, sort=False)

In [ ]:
popSurvival.dead10.value_counts(normalize=True)

In [ ]:
popSurvival.cvDeath10.value_counts(normalize=True)

In [ ]:
popSurvival.nonCVDeath10.value_counts(normalize=True)

In [ ]:
popSurvival.miDeath10.value_counts(normalize=True)

In [ ]:
popSurvival.strokeDeath10.value_counts(normalize=True)

In [ ]:
earlyCrossSectionOfNonImmigrants['ageDecile'] = pd.qcut(earlyCrossSectionOfNonImmigrants.age, 10)

mortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['diedBy2011','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
mortalityByDecileSim = popSurvival[['dead10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(mortalityByDecileSim.index.codes+0.5, mortalityByDecileSim.dead10, width=0.5, label="sim")
nhanesBar = plt.bar(mortalityByDecileNHANES.index.codes, mortalityByDecileNHANES.diedBy2011, width=0.5, label="nhanes")
plt.legend()


In [ ]:
strokeMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['strokeDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
mortalityByDecileSim = popSurvival[['strokeDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(mortalityByDecileSim.index.codes+0.5, mortalityByDecileSim.strokeDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(mortalityByDecileNHANES.index.codes, strokeMortalityByDecileNHANES.strokeDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
miMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['miDeath','ageDecile']].groupby('ageDecile').mean()
miMortalityByDecileSim = popSurvival[['miDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(miMortalityByDecileSim.index.codes+0.5, miMortalityByDecileSim.miDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(miMortalityByDecileNHANES.index.codes, miMortalityByDecileNHANES.miDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
cvMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['cvDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
cvMortalityByDecileSim = popSurvival[['cvDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

simBar =plt.bar(cvMortalityByDecileSim.index.codes+0.5, cvMortalityByDecileSim.cvDeath10, width=0.5, label="sim")
nhanesBar = plt.bar(cvMortalityByDecileNHANES.index.codes, cvMortalityByDecileNHANES.cvDeath, width=0.5, label="nhanes")
plt.legend()

In [ ]:
nonCVMortalityByDecileNHANES = earlyCrossSectionOfNonImmigrants[['nonCVDeath','ageDecile']].groupby(earlyCrossSectionOfNonImmigrants.ageDecile).mean()
nonCVMortalityByDecileSim = popSurvival[['nonCVDeath10', 'ageDecile']].groupby(popSurvival.ageDecile).mean()

nonCVsimBar =plt.bar(nonCVMortalityByDecileSim.index.codes+0.5, nonCVMortalityByDecileSim.nonCVDeath10, width=0.5, label="sim non cv")
cvSimBar =plt.bar(cvMortalityByDecileSim.index.codes+0.5, cvMortalityByDecileSim.cvDeath10, width=0.5, label="sim cv", bottom=nonCVMortalityByDecileSim.nonCVDeath10)

nonCVnhanesBar = plt.bar(nonCVMortalityByDecileNHANES.index.codes, nonCVMortalityByDecileNHANES.nonCVDeath, width=0.5, label="nhanes nonCV")
cvnhanesBar = plt.bar(cvMortalityByDecileNHANES.index.codes, cvMortalityByDecileNHANES.cvDeath, width=0.5, bottom=nonCVMortalityByDecileNHANES.nonCVDeath, label="nhanes CV")
plt.legend()

### So, we have slight under prediction of mortality in the simulation. Is that because of the code or the data?

To test — we will rerun the simulation on the direct NHANES data that we use in the calibration...

In [ ]:
from mcm.gender import NHANESGender
from mcm.person import Person
from mcm.outcome import Outcome
from mcm.race_ethnicity import NHANESRaceEthnicity
from mcm.smoking_status import SmokingStatus
from mcm.cohort_risk_model_repository import CohortRiskModelRepository
from mcm.outcome_model_repository import OutcomeModelRepository

In [ ]:
import os 

def calcMortalityRunningFullModel(x):
        temp =  Person(
            age=x.age,
            gender=NHANESGender(int(x.gender)),
            raceEthnicity=NHANESRaceEthnicity(int(x.raceEthnicity)),
            sbp=x.meanSBP,
            dbp=x.meanDBP,
            a1c=x.a1c,
            hdl=x.hdl,
            ldl=x.ldl,
            trig=x.trig,
            totChol=x.tot_chol,
            bmi=x.bmi,
            smokingStatus=SmokingStatus(int(x.smokingStatus)),
            dfIndex=x.index,
            diedBy2011=x.diedBy2011)
        
        for year in range(1,11):
            if not temp.is_dead():
                temp.advance_year(riskModel, cohortModel)
        return temp.is_dead()

earlyCrossSectionOfNonImmigrants['estMortalityFromModel'] = earlyCrossSectionOfNonImmigrants.apply(calcMortalityRunningFullModel, axis=1)

In [ ]:
earlyCrossSectionOfNonImmigrants['ageDecile'] = pd.qcut(earlyCrossSectionOfNonImmigrants.age, 10)
output = earlyCrossSectionOfNonImmigrants.groupby('ageDecile')[['diedBy2011', 'estMortalityFromModel']].mean()
print(output)
plt.bar(output.index.codes+1, output.diedBy2011, width=0.5, label="actualDeath")
plt.bar(output.index.codes+1.5, output.estMortalityFromModel, width=0.5, label="model")

plt.legend()

### Ok...this looks quite good...on the same data, the under-prediction problem looks very modest

In [ ]:
earlyCrossSectionOfNonImmigrants.to_stata("/Users/burke/Documents/research/bpCog/nhanes/earlyCrossSectionOfNonImmigrants.dta")
earlyCrossSectionOfNonImmigrants.ageDecile.value_counts().sort_index()

In [ ]:
earlyCrossSectionOfNonImmigrants[['diedBy2011', 'estMortalityFromCox', 'estMortalityFromModel']].mean()

In [ ]:
# well...this is not easy to explain.
# for the most part the code-bsaed calculation looks similar, gut it under predicts mortaltiy substantially in the highest age decile
# this is a bit strange...but, not impossible. if you look at the within population validation, it seems that we under-predicted in the oldest cohort for our basee model as well
# i wonder if having a longer follow-up (by constraining to the early oohort might cause that problem...that should be straight-forward to check)

# the code based model is over-estmiaging mortalty thorughout.
# this is going to be harder to figure otu...
# possibilities: 1. the times aren't right...and the cohort model is getting exposed to more time than the comparators
# the time aggregation isn't right...the one year probabilities are summing to something greater than the whole

In [ ]:
earlyCrossSectionOfNonImmigrants.ucod.value_counts()

In [ ]:
# next steps — 1. see if we under-preditx mortality badly in the NHANES popualtion if we look at only the people that are enrolled early (and thus have long follow-up)